In [ ]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = 'https://www.daraz.pk/'
driver.get(url)
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Redmi Note 12' + Keys.RETURN)
next_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div/div/ul/li[9]/a')

In [ ]:
import time
import pandas as pd
import numpy as np
data=dict()
data["descriptions"] = []
data["prices"]= []
data["rating"]=[]
data["free_shipping"]=[]
data["daraz_mall"]=[]
data["seller_rating"]=[]
data["ontime_rating"]=[]
data["score"]= []
#loop through pages 
for page in range(0,5):
    #find the ad grid and ad cards on the page
    ad_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

#loop through each ad card
    for i in ad_cards:
        try:
            #extract information from the ad card
            data["descriptions"].append(i.find_element(By.CLASS_NAME, 'title--wFj93').text)
            data["prices"].append(i.find_element(By.CLASS_NAME, 'price--NVB62').text)
            data["score"].append(0)
            data["free_shipping"].append(i.find_element(By.CLASS_NAME, 'location--eh0Ro').text)
            #check for Daraz Mall badge
            try:
                element = i.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]/div[39]/div/div/div[2]/div[1]/i')
                data["daraz_mall"].append(1)
            except NoSuchElementException:
                data["daraz_mall"].append(0)

            #click on the product link and switch to the new tab
            product_link_element = i.find_element(By.ID, 'id-a-link')
            product_link_element.send_keys(Keys.CONTROL + Keys.RETURN)
            driver.switch_to.window(driver.window_handles[1])
            #in present tab
            time.sleep(2)
            #extract information from the new tab
            percentage_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]')
            boxes=percentage_grid.find_elements(By.CLASS_NAME, 'info-content')
            #extract seller rating and on-time rating
            try:
                data["seller_rating"].append(boxes[0].find_element(By.CLASS_NAME, 'rating-positive').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["seller_rating"].append('0%')

            try:
                data["ontime_rating"].append(boxes[1].find_element(By.CLASS_NAME, 'seller-info-value').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["ontime_rating"].append('0%')
            
            #scroll down to load more content
            driver.execute_script("window.scrollTo(0,1800);")
            
            time.sleep(5)
            #extract overall rating
            try:
                rating_box=driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]/div/div/div[1]')
                data["rating"].append(rating_box.find_element(By.CLASS_NAME, 'score-average').text)
            except (NoSuchElementException,StaleElementReferenceException) as e :
                data["rating"].append("not found")

            time.sleep(2)
            #close the new tab and switch back to the original tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except(NoSuchElementException,StaleElementReferenceException) as e :
            print(e)
            #convert to dataframe and save to csv after every page
        df = pd.DataFrame(data)
        df.to_csv('Q3_data.csv')
        #click on next button to load page
    next_button.click()
    #add a delay to allow the new page to load
    time.sleep(5)
#     print(i)
#     print(page)



In [ ]:
#read saved csv
df=pd.read_csv("Q3_data.csv")

#iterate through each row in the dataFrame
for index, i in df.iterrows():
    #calculate score based on given formulas
    df.loc[index, 'score'] = (10000 * ((0.4 *((df.loc[index,'rating']) / 5)) +(0.4 * ((int(''.join(filter(str.isdigit, i['seller_rating'])))) / 100)) +(0.2 * ((int(''.join(filter(str.isdigit, i['ontime_rating'])))) / 100))))
     #add the prices to the score after extracting numerical values
    df.loc[index, 'score'] += int(''.join(filter(str.isdigit, i['prices'])))
    #add a bonus score if free shipping is available
    if(df.loc[index, 'free_shipping']=='Free Shipping'):
        df.loc[index, 'score'] += 1000
    #add a bonus score if the product is from daraz mall
    if(df.loc[index, 'daraz_mall']==1):
        df.loc[index, 'score'] += 5000   
df

In [ ]:
print('The best product is: ',df.loc[df['score'].idxmax()])

In [ ]:
df['descriptions'][40]